# Visual 3: Income and Wage Distribution Comparison (Austin TX and US)

This visualization highlights income inequality and the disconnect between rising home values and stagnant wages. It compares the distribution of household incomes in Austin with national averages, alongside a wage distribution comparison for Texas versus the U.S. Together, these views demonstrate that Austin households are concentrated in higher income brackets, while Texas workers overall are more concentrated in lower wage range, suggesting a local affordability divide reinforced by uneven economic growth.

In [ ]:
%pip install pandas plotly

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [ ]:
# Load household income data
income_df = pd.read_csv('data files/austin_household_income.csv')

# Load wage distribution data
wage_df = pd.read_csv('data files/texas_wage_distribution.csv')

print("Income data years:", sorted(income_df['Year'].unique()))
print("Wage data years:", sorted(wage_df['Year'].unique()))
print("\nIncome data shape:", income_df.shape)
print("Wage data shape:", wage_df.shape)

In [ ]:
# Find common years between both datasets
income_years = set(income_df['Year'].unique())
wage_years = set(wage_df['Year'].unique())
common_years = sorted(list(income_years.intersection(wage_years)))
print(f"Common years: {common_years}")

# Filter to common years
income_filtered = income_df[income_df['Year'].isin(common_years)].copy()
wage_filtered = wage_df[wage_df['Year'].isin(common_years)].copy()

# Separate Austin and US income data
austin_income = income_filtered[income_filtered['Place'] == 'Austin, TX'].copy()
us_income = income_filtered[income_filtered['Place'] == 'United States'].copy()

# Separate Texas and US wage data
texas_wage = wage_filtered[wage_filtered['State'] == 'Texas'].copy()
us_wage = wage_filtered[wage_filtered['State'] == 'United States'].copy()

print(f"\nAustin income records: {len(austin_income)}")
print(f"US income records: {len(us_income)}")
print(f"Texas wage records: {len(texas_wage)}")
print(f"US wage records: {len(us_wage)}")

In [ ]:
# Define colors - warm orange for Austin/Texas, cool gray-blue for US
austin_texas_color = '#FF8C42'  # Warm orange
us_color = '#6B8CAE'  # Cool gray-blue

# Create subplots with two panels
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        'Household Income Distribution (Austin vs. U.S.)',
        'Wage Distribution (Texas vs. U.S.)'
    ),
    horizontal_spacing=0.15
)

# Create frames for each year (for the slider)
frames = []

for year in common_years:
    # Filter data for the current year
    austin_year = austin_income[austin_income['Year'] == year].sort_values('Household Income Bucket ID')
    us_income_year = us_income[us_income['Year'] == year].sort_values('Household Income Bucket ID')
    texas_year = texas_wage[texas_wage['Year'] == year].sort_values('Wage Bin ID')
    us_wage_year = us_wage[us_wage['Year'] == year].sort_values('Wage Bin ID')
    
    frame_data = [
        # Austin income bars
        go.Bar(
            x=austin_year['Household Income Bucket'],
            y=austin_year['share'] * 100,
            name='Austin, TX',
            marker_color=austin_texas_color,
            legendgroup='income',
            showlegend=True,
            hovertemplate='<b>%{x}</b><br>' +
                          'Austin: %{y:.2f}%<br>' +
                          f'Year: {year}<extra></extra>'
        ),
        # US income bars
        go.Bar(
            x=us_income_year['Household Income Bucket'],
            y=us_income_year['share'] * 100,
            name='United States',
            marker_color=us_color,
            legendgroup='income',
            showlegend=True,
            hovertemplate='<b>%{x}</b><br>' +
                          'U.S.: %{y:.2f}%<br>' +
                          f'Year: {year}<extra></extra>'
        ),
        # Texas wage bars
        go.Bar(
            x=texas_year['Wage Bin'],
            y=texas_year['share'] * 100,
            name='Texas',
            marker_color=austin_texas_color,
            legendgroup='wage',
            showlegend=True,
            hovertemplate='<b>%{x}</b><br>' +
                          'Texas: %{y:.2f}%<br>' +
                          f'Year: {year}<extra></extra>'
        ),
        # US wage bars
        go.Bar(
            x=us_wage_year['Wage Bin'],
            y=us_wage_year['share'] * 100,
            name='United States',
            marker_color=us_color,
            legendgroup='wage',
            showlegend=True,
            hovertemplate='<b>%{x}</b><br>' +
                          'U.S.: %{y:.2f}%<br>' +
                          f'Year: {year}<extra></extra>'
        )
    ]
    
    frames.append(go.Frame(
        data=frame_data,
        name=str(year),
        layout=go.Layout(
            annotations=[
                # Annotation for income panel
                dict(
                    text="Austin's higher $200k+ share<br>reflects wealth concentration",
                    xref="x",
                    yref="y",
                    x=15,  # Position near the $200k+ bucket (last bucket)
                    y=20,
                    showarrow=True,
                    arrowhead=2,
                    ax=50,
                    ay=-40,
                    font=dict(size=10, color='#333'),
                    bgcolor='rgba(255, 255, 255, 0.8)',
                    borderpad=4
                ),
                # Annotation for wage panel
                dict(
                    text="Texas wages are more concentrated<br>in lower brackets",
                    xref="x2",
                    yref="y2",
                    x=3,  # Position near lower wage brackets
                    y=18,
                    showarrow=True,
                    arrowhead=2,
                    ax=-50,
                    ay=-40,
                    font=dict(size=10, color='#333'),
                    bgcolor='rgba(255, 255, 255, 0.8)',
                    borderpad=4
                )
            ]
        )
    ))

# Add initial data (first year)
first_year = common_years[0]
austin_first = austin_income[austin_income['Year'] == first_year].sort_values('Household Income Bucket ID')
us_income_first = us_income[us_income['Year'] == first_year].sort_values('Household Income Bucket ID')
texas_first = texas_wage[texas_wage['Year'] == first_year].sort_values('Wage Bin ID')
us_wage_first = us_wage[us_wage['Year'] == first_year].sort_values('Wage Bin ID')

# Add income bars to left panel (col 1)
fig.add_trace(
    go.Bar(
        x=austin_first['Household Income Bucket'],
        y=austin_first['share'] * 100,
        name='Austin, TX',
        marker_color=austin_texas_color,
        legendgroup='income',
        hovertemplate='<b>%{x}</b><br>' +
                      'Austin: %{y:.2f}%<br>' +
                      f'Year: {first_year}<extra></extra>'
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=us_income_first['Household Income Bucket'],
        y=us_income_first['share'] * 100,
        name='United States',
        marker_color=us_color,
        legendgroup='income',
        hovertemplate='<b>%{x}</b><br>' +
                      'U.S.: %{y:.2f}%<br>' +
                      f'Year: {first_year}<extra></extra>'
    ),
    row=1, col=1
)

# Add wage bars to right panel (col 2)
fig.add_trace(
    go.Bar(
        x=texas_first['Wage Bin'],
        y=texas_first['share'] * 100,
        name='Texas',
        marker_color=austin_texas_color,
        legendgroup='wage',
        hovertemplate='<b>%{x}</b><br>' +
                      'Texas: %{y:.2f}%<br>' +
                      f'Year: {first_year}<extra></extra>'
    ),
    row=1, col=2
)

fig.add_trace(
    go.Bar(
        x=us_wage_first['Wage Bin'],
        y=us_wage_first['share'] * 100,
        name='United States',
        marker_color=us_color,
        legendgroup='wage',
        hovertemplate='<b>%{x}</b><br>' +
                      'U.S.: %{y:.2f}%<br>' +
                      f'Year: {first_year}<extra></extra>'
    ),
    row=1, col=2
)

# Add frames to figure
fig.frames = frames

# Create slider
sliders = [dict(
    active=0,
    yanchor="top",
    y=-0.15,
    xanchor="left",
    currentvalue=dict(
        prefix="Year: ",
        visible=True,
        xanchor="left",
        font=dict(size=16)
    ),
    pad=dict(b=10, t=30),
    len=0.9,
    x=0.05,
    steps=[dict(
        args=[[str(year)],
              dict(frame=dict(duration=300, redraw=True),
                   mode="immediate",
                   transition=dict(duration=300))],
        label=str(year),
        method="animate"
    ) for year in common_years]
)]

# Update layout
fig.update_layout(
    title=dict(
        text='Income and Wage Distribution Comparison: Austin/Texas vs. U.S.',
        font=dict(size=18, color='#333'),
        x=0.5,
        xanchor='center'
    ),
    height=600,
    showlegend=True,
    barmode='group',
    sliders=sliders,
    legend=dict(
        title=dict(text='Region', font=dict(size=14)),
        orientation='h',
        yanchor='bottom',
        y=-0.25,
        xanchor='center',
        x=0.5,
        font=dict(size=12)
    ),
    hovermode='closest',
    margin=dict(b=150, t=100)
)

# Update axes
fig.update_xaxes(
    title_text="Income Bracket",
    tickangle=-45,
    tickfont=dict(size=9),
    row=1, col=1
)

fig.update_xaxes(
    title_text="Wage Bracket",
    tickangle=-45,
    tickfont=dict(size=9),
    row=1, col=2
)

fig.update_yaxes(
    title_text="Share of Households (%)",
    row=1, col=1
)

fig.update_yaxes(
    title_text="Share of Workers (%)",
    row=1, col=2
)

# Add annotations for initial year
fig.add_annotation(
    text="Austin's higher $200k+ share<br>reflects wealth concentration",
    xref="x",
    yref="y",
    x=15,  # Position near the $200k+ bucket (last bucket)
    y=20,
    showarrow=True,
    arrowhead=2,
    ax=50,
    ay=-40,
    font=dict(size=10, color='#333'),
    bgcolor='rgba(255, 255, 255, 0.8)',
    borderpad=4
)

fig.add_annotation(
    text="Texas wages are more concentrated<br>in lower brackets",
    xref="x2",
    yref="y2",
    x=3,  # Position near lower wage brackets
    y=18,
    showarrow=True,
    arrowhead=2,
    ax=-50,
    ay=-40,
    font=dict(size=10, color='#333'),
    bgcolor='rgba(255, 255, 255, 0.8)',
    borderpad=4
)

# Show the figure
fig.show()

## Key Insights

This visualization reveals important economic disparities:

1. **Income Concentration**: Austin households show a higher concentration in the $200k+ income bracket compared to national averages, indicating wealth concentration in the city.

2. **Wage Disparity**: Texas workers are more heavily concentrated in lower wage brackets compared to the U.S. average, suggesting that many workers earn lower wages.

3. **Affordability Divide**: The combination of high-income concentration and lower wages creates an affordability divide. While some Austin households have high incomes that can support rising home prices, many Texas workers struggle with stagnant wages, making homeownership increasingly difficult.

4. **Economic Growth Inequality**: The data suggests uneven economic growth, where benefits accrue disproportionately to higher-income households while lower-wage workers see limited wage growth.

Use the year slider above to explore how these patterns have evolved over time.